A API do Google https://ai.google.dev/pricing?hl=pt-br#1_5flash aqui mostra os limites gratuitos

Este jupyter notebook tem por objetivo auxiliar a entidade DACES-UNB,no intuito de acelerar processos internos de SCAN OCR de maneira mais inteligente, garantindo precisão e menos trabalho físico para o processo de scan de livros, facilitando a vida da equipe DACES.

In [5]:
# Instalação das dependências necessárias
!pip install pytesseract opencv-python easyocr transformers spacy pillow
!python -m spacy download pt_core_news_sm
!pip install git+https://github.com/lukas-blecher/LaTeX-OCR.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.1 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
  Cloning https://github.com/lukas-blecher/LaTeX-OCR.git to /tmp/pip-req-build-9epktsc9
  Running command git clone --filter=blob:none --quiet https://github.com/lukas-blecher/LaTeX-OCR.git /tmp/pip-req-build-9epktsc9
  Resolved https://github.com/lukas-blecher/LaTeX-OCR.git to commit 5c1ac929bd19a7ecf86d5fb8d94771c8969fcb80
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached timm-0.5.4-py3-none-any.whl.metadata (36 kB)
Using cached timm-0.5.4-py3-none-any.whl (431 kB)
  Created wheel for pix2tex: filename=pix2tex-0.1.4-py3-none-any.whl size=423426 sha256=cba66e9bd5f389ad6292e68bd0c9168ed53c9dcfe7447f54b8a2877cb3717d62
  Stored in directory: /tmp/pip-ephem-wheel-cache-7jgqy4op/wheels/3f/0f/4

In [6]:
!pip install tiktoken
!pip install protobuf
!pip install sentencepiece

In [7]:
# Importações
import cv2
import pytesseract
import easyocr
import numpy as np
from transformers import pipeline
from PIL import Image
import spacy
from difflib import SequenceMatcher
import re
from collections import Counter
from PIL import Image
from pix2tex.cli import LatexOCR

KeyboardInterrupt: 

In [ ]:
class OCREngine:
    def __init__(self):
        """Inicializa os modelos e configurações necessárias"""
        # EasyOCR para português
        self.reader = easyocr.Reader(['pt'])
        # Pipeline do transformers para mT5
        self.nlp_transformer = pipeline("text2text-generation", model="google/mt5-small")
        # Modelo spaCy para português
        self.nlp_spacy = spacy.load('pt_core_news_sm')

    def convert_png_to_various_formats(self, png_path):
        """Converte a imagem para diferentes formatos para processamento"""
        original = cv2.imread(png_path)

        if original is None:
            raise ValueError(f"Não foi possível ler a imagem no caminho: {png_path}")

        formats = {
            'original': original,
            'grayscale': cv2.cvtColor(original, cv2.COLOR_BGR2GRAY),
            'binary': cv2.threshold(
                cv2.cvtColor(original, cv2.COLOR_BGR2GRAY),
                127, 255, cv2.THRESH_BINARY
            )[1],
            'adaptive': cv2.adaptiveThreshold(
                cv2.cvtColor(original, cv2.COLOR_BGR2GRAY),
                255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
            ),
            'denoised': cv2.fastNlMeansDenoising(
                cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
            )
        }

        return formats

    def pix_to_text(self, image):
        """Lê as equações do texto"""
        try:
            text = pytesseract.image_to_string(image, config='--psm 6')
            return text.strip()
        except:
            return ""

    def tesseract_ocr(self, image):
        """Extrai texto usando Tesseract"""
        try:
            text = pytesseract.image_to_string(image, lang='por')
            return text.strip()
        except:
            return ""

    def easyocr_process(self, image):
        """Extrai texto usando EasyOCR"""
        try:
            results = self.reader.readtext(image)
            return " ".join([result[1] for result in results])
        except:
            return ""

    def process_with_mt5(self, text):
        """Melhora o texto usando mT5"""
        try:
            improved = self.nlp_transformer(text[:500], max_length=500)[0]['generated_text']
            return improved
        except:
            return text

    def calculate_text_coherence(self, text):
        """Calcula coerência do texto baseado em múltiplos fatores"""
        if not text.strip():
            return 0

        doc = self.nlp_spacy(text)

        # Métricas de coerência
        scores = {
            'words_ratio': len([token for token in doc if token.is_alpha]) / max(len(text.split()), 1),
            'sentences_ratio': len(list(doc.sents)) / max(len(text.split()), 1),
            'stopwords_ratio': len([token for token in doc if token.is_stop]) / max(len(doc), 1),
            'punctuation_ratio': len([token for token in doc if token.is_punct]) / max(len(doc), 1)
        }

        # Peso para cada métrica
        weights = {
            'words_ratio': 0.4,
            'sentences_ratio': 0.3,
            'stopwords_ratio': 0.2,
            'punctuation_ratio': 0.1
        }

        return sum(score * weights[metric] for metric, score in scores.items())

    def similarity_between_texts(self, text1, text2):
        """Calcula similaridade entre dois textos"""
        return SequenceMatcher(None, text1, text2).ratio()

    def combine_results(self, results):
        """Combina resultados usando análise de coerência e similaridade"""
        texts = {k: v for k, v in results.items() if v.strip()}
        if not texts:
            return ""

        # Calcula score de coerência para cada texto
        coherence_scores = {
            method: self.calculate_text_coherence(text)
            for method, text in texts.items()
        }

        # Calcula similaridade entre os textos
        similarities = {}
        for method1, text1 in texts.items():
            for method2, text2 in texts.items():
                if method1 < method2:
                    key = f"{method1}_{method2}"
                    similarities[key] = self.similarity_between_texts(text1, text2)

        # Encontra o texto mais confiável
        best_method = max(coherence_scores.items(), key=lambda x: x[1])[0]
        best_text = texts[best_method]

        # Análise detalhada
        print("\nAnálise de qualidade dos textos:")
        print("\nScores de coerência:")
        for method, score in coherence_scores.items():
            print(f"{method}: {score:.3f}")

        print("\nSimilaridades entre textos:")
        for key, sim in similarities.items():
            print(f"{key}: {sim:.3f}")

        print(f"\nMétodo escolhido: {best_method}")

        return best_text

    def process_image(self, png_path):
        """Processo principal de OCR"""
        # Converte imagem para diferentes formatos
        formats = self.convert_png_to_various_formats(png_path)

        # Extrai texto usando diferentes métodos
        results = {
            'tesseract_original': self.tesseract_ocr(formats['original']),
            'tesseract_processed': self.tesseract_ocr(formats['adaptive']),
            'easyocr': self.easyocr_process(formats['original'])
        }

        # Combina e melhora os resultados
        final_text = self.combine_results(results)
        if final_text:
            final_text = self.process_with_mt5(final_text)

        return {
            'individual_results': results,
            'combined_result': final_text
        }

    def process_multiple_images(self, image_pattern, pages='all'):
        """
        Processa múltiplas imagens PNG.

        Args:
            image_pattern (str): Padrão do nome dos arquivos (ex: 'pagina_{}.png')
            pages: Pode ser:
                - 'all' para todas as imagens encontradas
                - número inteiro para uma imagem específica
                - string no formato '3-10' para um range
                - lista de números para imagens específicas [1,3,5]

        Returns:
            dict: Resultados do OCR por imagem
        """
        try:
            # Determina quais páginas processar
            if pages == 'all':
                # Encontra todas as imagens que seguem o padrão
                all_files = glob.glob(image_pattern.format('*'))
                pages_to_process = list(range(1, len(all_files) + 1))

            elif isinstance(pages, int):
                # Processa uma página específica
                pages_to_process = [pages]

            elif isinstance(pages, str) and '-' in pages:
                # Processa um range de páginas
                start, end = map(int, pages.split('-'))
                pages_to_process = list(range(start, end + 1))

            elif isinstance(pages, list):
                # Processa páginas específicas da lista
                pages_to_process = pages

            else:
                raise ValueError("Formato de páginas inválido")

            # Processa cada imagem
            results = {}
            for page_num in pages_to_process:
                image_path = image_pattern.format(page_num)

                if not os.path.exists(image_path):
                    print(f"Aviso: Imagem {image_path} não encontrada")
                    continue

                print(f"Processando imagem {page_num}...")
                page_result = self.process_image(image_path)
                results[page_num] = page_result

            return results

        except Exception as e:
            raise Exception(f"Erro ao processar imagens: {str(e)}")

In [ ]:
!rm -rf semana_07ex
!unzip Math.zip

In [ ]:
import os

folder_path = '/content/semana_07ex'
filenames = sorted(os.listdir(folder_path))  # Sort filenames
file_count = 1

for filename in filenames:
    source_path = os.path.join(folder_path, filename)
    new_filename = f"pagina_{file_count}.png"
    destination_path = os.path.join(folder_path, new_filename)
    os.rename(source_path, destination_path)
    file_count += 1

print("Files renamed successfully!")

In [ ]:
import os
import glob

def main():

    """Função principal que salva os resultados em arquivos txt"""
    engine = OCREngine()

    # Configurações - ajuste estes parâmetros conforme necessário
    pasta_entrada = "assets/semana_07ex"  # Coloque aqui o caminho da pasta com as imagens
    # pages = 3  # Pode ser: 'all', número específico, 'inicio-fim', ou lista [1,2,3]
    pages = 'all'
    # pages = '3 - 10'
    # pages = [1,3,5,7]

    # Ajusta o padrão para incluir o caminho completo
    image_pattern = os.path.join(pasta_entrada, 'pagina_{}.png')

    # Cria diretório para resultados se não existir
    if not os.path.exists('resultados'):
        os.makedirs('resultados')

    # Processa as imagens
    results = engine.process_multiple_images(image_pattern, pages)

    # Salva os resultados
    for page_num, result in results.items():
        filename = f'resultados/pagina_{page_num}_resultados.txt'

        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"=== Resultados da Imagem {page_num} ===\n\n")

            f.write("=== RESULTADO COMBINADO ===\n")
            f.write(result['combined_result'])
            f.write("\n\n")

            f.write("=== RESULTADOS INDIVIDUAIS ===\n")
            for method, text in result['individual_results'].items():
                f.write(f"\n--- {method} ---\n")
                f.write(text)
                f.write("\n")

    print(f"Resultados salvos na pasta 'resultados'")

if __name__ == "__main__":
    main()